In [62]:
import pandas as pd

In [63]:
df = pd.read_csv("data.csv")

- first country
- duration in min
- remove empty budgets, revenue
- budget starts with $
- genres: drama, comedy, horror, docu, thriller, action, romance, western

In [64]:
df

,imdb_id,title,genres,release_date,country,production_companies,budget,revenue,duration,actors,director
0,114709,Toy Story (1995),Adventure;Animation;Children;Comedy;Fantasy,"Release dateNovember 22, 1995 (United States)",Country of originUnited States,Walt Disney Pictures;Pixar Animation Studios,"$30,000,000 (estimated)","$29,140,617",1 hr 21 min,Tom Hanks;Tim Allen;Don Rickles;Jim Varney;Wal...,John Lasseter
1,113497,Jumanji (1995),Adventure;Children;Fantasy,"Release dateDecember 15, 1995 (United States)",Country of originUnited States,TriStar Pictures;Interscope Communications;Tei...,"$65,000,000 (estimated)","$11,084,370",1 hr 44 min,Robin Williams;Kirsten Dunst;Bonnie Hunt;Jonat...,Joe Johnston
2,113228,Grumpier Old Men (1995),Comedy;Romance,"Release dateDecember 22, 1995 (United States)",Country of originUnited States,Lancaster Gate;Warner Bros.,"$25,000,000 (estimated)","$7,797,185",1 hr 41 min,Walter Matthau;Jack Lemmon;Ann-Margret;Sophia ...,Howard Deutch
3,114885,Waiting to Exhale (1995),Comedy;Drama;Romance,"Release dateDecember 22, 1995 (United States)",Country of originUnited States,Twentieth Century Fox,"$16,000,000 (estimated)","$14,126,927",2 hr 4 min,Whitney Houston;Angela Bassett;Loretta Devine;...,Forest Whitaker
4,113041,Father of the Bride Part II (1995),Comedy,"Release dateDecember 8, 1995 (United States)",Country of originUnited States,Touchstone Pictures;Sandollar Productions;Tayl...,"$30,000,000 (estimated)","$11,134,978",1 hr 46 min,Steve Martin;Diane Keaton;Martin Short;Kimberl...,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...
20626,1801061,Me and you (io e te) (2012),Drama,"Release dateOctober 25, 2012 (Italy)",Country of originItaly,Fiction;Wildside;Medusa Film,NaN,NaN,1 hr 43 min,Jacopo Olmo Antinori;Tea Falco;Sonia Bergamasc...,Bernardo Bertolucci
20627,64725,Night Gallery (1969),Horror;Mystery,"Release dateNovember 8, 1969 (United States)",Country of originUnited States,Universal Television,NaN,NaN,1 hr 38 min,Joan Crawford;Ossie Davis;Richard Kiley;Roddy ...,Boris Sagal(segment The Cemetery);Barry Shear(...
20628,1747958,Blood Ties (2013),Crime;Drama;Thriller,"Release dateOctober 30, 2013 (France)",Countries of originFranceUnited States,Trésor Films;Worldview Entertainment;Caneo Films,"$25,500,000 (estimated)","$26,912",2 hr 7 min,Clive Owen;Marion Cotillard;Billy Crudup;Mila ...,Guillaume Canet
20629,3203290,Free to Play (2014),Documentary,"Release dateApril 14, 2021 (Norway)",Country of originUnited States,Valve,"$150,000 (estimated)",NaN,1 hr 15 min,Benedict Lim;Clinton Loomis;Danil Ishutin;Edwi...,Valve


In [65]:
df_small = df.dropna(
    subset=["country", "budget", "revenue", "duration"]
)
df_small = df_small[df_small["genres"].isin(["Comedy", "Drama", "Horror", "Documentary", "Thriller", "Action", "Romance", "Western"])]

In [66]:
df_small["month"] = df_small["release_date"].str.extract(r"(January|February|March|April|May|June|July|August|September|October|November|December)")

In [67]:
df_small["date"] = df_small["release_date"].str.extract(r"(\d\d?)")

In [68]:
df_small["year"] = df_small["release_date"].str.extract(r"(\d{4})")

In [69]:
df_small["budget"] = df_small["budget"].str.extract(r"\$([\d,]+)")
df_small = df_small.dropna(subset=["budget"])
df_small["budget"] = df_small["budget"].astype(str).apply(lambda b: int(b.replace(",", "")))

In [70]:
df_small["revenue"] = df_small["revenue"].str.extract(r"([\d,]+)")
df_small = df_small.dropna(subset=["revenue"])
df_small["revenue"] = df_small["revenue"].astype(str).apply(lambda r: int(r.replace(",", "")))

In [71]:
import re
def convert_duration_to_min(d):
    times = re.findall("\d+", d)
    hr, minut = 0, 0
    if len(times) == 2:
        hr, minut = times
    elif "hr" in d:
        hr = times[0]
    elif "min" in d:
        minut = times[0]
    return int(hr) * 60 + int(minut)

In [72]:
df_small["duration"] = df_small["duration"].apply(convert_duration_to_min)

In [73]:
def get_main_country(countrystring):
    dual = False
    for i in range(len(list(countrystring))-1):
        if list(countrystring)[i].islower() and list(countrystring)[i+1].isupper():
            dual = True
    if dual == True:
        return re.search("(.+?[a-z])(?=[A-Z])",countrystring).group(0)
    else:
        return countrystring

In [74]:
df_small["country"] = (
    df_small["country"].str.replace("Country of origin", "")
    .str.replace("Countries of origin", "")
    .apply(get_main_country)
    .apply(lambda c: 1 if c != "United States" else 0)
)

In [75]:
df = df.rename({"country": "is_foreign"}, axis=1)

In [76]:
df_small = df_small.drop("release_date", axis=1)

In [77]:
df_small.shape

(1420, 13)

In [78]:
df_small["genres"].value_counts()

Drama          629
Comedy         519
Horror          78
Documentary     69
Thriller        60
Action          35
Romance         17
Western         13
Name: genres, dtype: int64

In [79]:
df_small.reset_index(drop=True).to_csv("data_clean.csv")

In [80]:
df = pd.read_csv("data_clean.csv", index_col=0)

In [81]:
df

,imdb_id,title,genres,country,production_companies,budget,revenue,duration,actors,director,month,date,year
0,113041,Father of the Bride Part II (1995),Comedy,0,Touchstone Pictures;Sandollar Productions;Tayl...,30000000,11134978,106,Steve Martin;Diane Keaton;Martin Short;Kimberl...,Charles Shyer,December,8,1995
1,113987,Nixon (1995),Drama,0,Hollywood Pictures;Illusion Entertainment;Cine...,44000000,2206506,192,Anthony Hopkins;Joan Allen;Powers Boothe;Ed Ha...,Oliver Stone,January,5,1996
2,112281,Ace Ventura: When Nature Calls (1995),Comedy,0,Morgan Creek Entertainment;Morgan Creek Produc...,30000000,37804076,90,Jim Carrey;Ian McNeice;Simon Callow;Maynard Ez...,Steve Oedekerk,November,10,1995
3,114057,Othello (1995),Drama,0,Castle Rock Entertainment;Dakota Films;Imminen...,11000000,29097,123,Laurence Fishburne;Kenneth Branagh;Irène Jacob...,Oliver Parker,January,19,1996
4,112792,Dangerous Minds (1995),Drama,0,Hollywood Pictures;Don Simpson/Jerry Bruckheim...,23000000,14931503,99,Michelle Pfeiffer;George Dzundza;Courtney B. V...,John N. Smith,August,11,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,2382396,Joe (2013),Drama,0,Worldview Entertainment;Dreambridge Films;Musk...,4000000,105881,117,Nicolas Cage;Tye Sheridan;Gary Poulter;Ronnie ...,David Gordon Green,April,11,2014
1416,2363471,"Summit, The (2012)",Documentary,1,Image Now;Fantastic Films;Passion Pictures,1000000,20904,95,Christine Barnes;Hoselito Bite;Walter Bonatti;...,Nick Ryan,November,22,2013
1417,1992193,Pandora's Promise (2013),Documentary,0,Robert Stone Productions;Vulcan Productions,1000000,23419,87,Stewart Brand;Richard Rhodes;Gwyneth Cravens;M...,Robert Stone,November,15,2013
1418,2170299,Bad Words (2013),Comedy,0,Aggregate Films;Darko Entertainment;Digital Im...,10000000,113301,89,Jason Bateman;Kathryn Hahn;Allison Janney;Roha...,Jason Bateman,March,28,2014
